In [1]:
pip install openai

  Using cached openai-1.93.0-py3-none-any.whl.metadata (29 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.10.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (5.2 kB)
Using cached openai-1.93.0-py3-none-any.whl (755 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached jiter-0.10.0-cp311-cp311-macosx_11_0_arm64.whl (321 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [openai]2m2/3 [openai]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import asyncio
import os
from PyPDF2 import PdfReader
from neo4j import GraphDatabase
import xml.etree.ElementTree as ET
import logging 
import sys
from lightrag import LightRAG, QueryParam
from lightrag.llm.ollama import ollama_model_complete, ollama_embed
from lightrag.kg.shared_storage import initialize_pipeline_status
from lightrag.utils import EmbeddingFunc, logger, set_verbose_debug
from openai import AsyncOpenAI
from prompt_it import PROMPTS_IT
PDF_FOLDER = "docs"
WORKING_DIR = "./pa_rag_storage" 

# --- Ollama Model Configuration ---
OLLAMA_GENERATION_MODEL = "gemma3:1b"
OLLAMA_EMBEDDING_MODEL = "nomic-embed-text"
OLLAMA_EMBEDDING_DIM = 768 

LM_STUDIO_API_BASE = "http://localhost:1234/v1"
LM_STUDIO_MODEL_NAME = "google/gemma-3-1b" 

EMBEDDING_MODEL = "multilingual-e5-large-instruct-GGUF/multilingual-e5-large-instruct-q8_0.gguf"
EMBEDDING_DIM = 768
# --- Neo4j Database Configuration ---
os.environ["NEO4J_URI"] = "neo4j://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "giacomo3234" 

lm_studio_client = AsyncOpenAI(base_url=LM_STUDIO_API_BASE, api_key="not-needed")

os.makedirs(WORKING_DIR, exist_ok=True)

print("Configuration and corrected imports are set.")

# Get the root logger for the 'lightrag' library
lightrag_logger = logging.getLogger("lightrag")

# Set the logging level to INFO or DEBUG
# - INFO: Shows major steps (e.g., "Processing chunk 1/10", "Extracting entities"). This is a good default.
# - DEBUG: Shows extremely detailed information, including the exact prompts being sent to the LLM. Very useful for debugging.
lightrag_logger.setLevel(logging.INFO)

# Make sure the logs are printed to your notebook's output
# We check if a handler already exists to avoid duplicate messages
if not any(isinstance(handler, logging.StreamHandler) for handler in lightrag_logger.handlers):
    handler = logging.StreamHandler(sys.stdout)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    lightrag_logger.addHandler(handler)

print("Logging configured. You will now see detailed progress from the LightRAG library.")

2025-07-07 15:59:56 - pipmaster.package_manager - INFO - Targeting pip associated with Python: /Users/giacomo/Documents/kg+llm_task2_nlp/.conda/bin/python | Command base: /Users/giacomo/Documents/kg+llm_task2_nlp/.conda/bin/python -m pip
2025-07-07 15:59:56 - pipmaster.package_manager - INFO - Targeting pip associated with Python: /Users/giacomo/Documents/kg+llm_task2_nlp/.conda/bin/python | Command base: /Users/giacomo/Documents/kg+llm_task2_nlp/.conda/bin/python -m pip
2025-07-07 15:59:56 - pipmaster.async_package_manager - INFO - [Async] Initialized for Python: /Users/giacomo/Documents/kg+llm_task2_nlp/.conda/bin/python


Configuration and corrected imports are set.
Logging configured. You will now see detailed progress from the LightRAG library.


In [ ]:
# --- Helper function to read PDFs ---
def get_text_from_pdfs(folder_path):
    all_text = ""
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.pdf'):
            print(f"Reading PDF: {filename}")
            with open(os.path.join(folder_path, filename), 'rb') as f:
                reader = PdfReader(f)
                for page in reader.pages:
                    all_text += page.extract_text() + "\n"
    return all_text

async def lm_studio_generator(prompt: str, **kwargs) -> str:
    """An async wrapper for the LM Studio OpenAI-compatible API."""
    try:
        # --- THIS IS THE FIX ---
        # The lightrag library passes internal arguments like 'hashing_kv'.
        # The OpenAI client doesn't recognize this, so we safely remove it
        # from the kwargs dictionary before making the API call.
        kwargs.pop('hashing_kv', None)

        response = await lm_studio_client.chat.completions.create(
            model=LM_STUDIO_MODEL_NAME,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            # Now, **kwargs only contains arguments the client understands
            **kwargs
        )
        return response.choices[0].message.content or ""
    except Exception as e:
        # This will now also catch timeout errors if you add them
        print(f"Error during LM Studio generation: {e}")
        return ""
    

async def run_lightrag_ingestion():
    embedding_function_wrapper = EmbeddingFunc(
        embedding_dim=OLLAMA_EMBEDDING_DIM,
        max_token_size=4096,
        func=lambda texts: ollama_embed(texts, embed_model=OLLAMA_EMBEDDING_MODEL)
    )
    


    rag_instance = LightRAG(
        working_dir=WORKING_DIR,
        llm_model_func=lm_studio_generator,
        #llm_model_max_async=1,
        embedding_func=embedding_function_wrapper,
        graph_storage="Neo4JStorage",
        chunk_token_size=512,  
        chunk_overlap_token_size=50,
        addon_params={
        "language": PROMPTS_IT["DEFAULT_LANGUAGE"],
        "entity_types": PROMPTS_IT["DEFAULT_ENTITY_TYPES"],
        "entity_extraction_prompt": PROMPTS_IT["entity_extraction"],
        "entity_extraction_examples": PROMPTS_IT["entity_extraction_examples"]
        }
    )


    print("--- Initializing LightRAG Storages (connecting to Neo4j...) ---")
    await rag_instance.initialize_storages()
    await initialize_pipeline_status()

    print("--- Reading PA Regulation PDFs ---")
    document_text = get_text_from_pdfs(PDF_FOLDER)
    if not document_text:
        print("No text found. Halting.")
        return

    documents_to_insert = [document_text]
    document_ids = ["pa_regulations_corpus_lmstudio"]
    
    print(f"\n--- Ingesting document with ID '{document_ids[0]}' into LightRAG... ---")
    await rag_instance.ainsert(documents_to_insert, ids=document_ids)
    
    print("\n--- Ingestion Complete! ---")
    print("Your knowledge graph has been generated and loaded directly into Neo4j.")

# Run the asynchronous ingestion process
await run_lightrag_ingestion()

INFO: Process 3777 Shared-Data already initialized (multiprocess=False)
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './pa_rag_storage/vdb_entities.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './pa_rag_storage/vdb_relationships.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './pa_rag_storage/vdb_chunks.json'} 0 data
INFO: Process 3777 storage namespace already initialized: [full_docs]
INFO: Process 3777 storage namespace already initialized: [text_chunks]
INFO: Process 3777 storage namespace already initialized: [llm_response_cache]
INFO: Process 3777 storage namespace already initialized: [doc_status]
INFO: Process 3777 storage namespace already initialized: [full_docs]
INFO: Process 3777 storage namespace already initialized: [text_chunks]
INFO: Process 3777 storage namespace already initialized: [llm_response_cache]
INFO: Process 3777 storage n

--- Initializing LightRAG Storages (connecting to Neo4j...) ---
2025-07-07 16:02:54,359 - lightrag - INFO - Chunk-entity-relation at neo4j://localhost:7687 not found. try to create specified database.
2025-07-07 16:02:54,360 - lightrag - INFO - Chunk-entity-relation at neo4j://localhost:7687 not found. try to create specified database.
2025-07-07 16:02:54,389 - lightrag - WARNING - This Neo4j instance does not support creating databases. Try to use Neo4j Desktop/Enterprise version or DozerDB instead. Fallback to use the default database.
2025-07-07 16:02:54,390 - lightrag - WARNING - This Neo4j instance does not support creating databases. Try to use Neo4j Desktop/Enterprise version or DozerDB instead. Fallback to use the default database.
2025-07-07 16:02:54,404 - lightrag - INFO - Connected to None at neo4j://localhost:7687
2025-07-07 16:02:54,405 - lightrag - INFO - Connected to None at neo4j://localhost:7687
2025-07-07 16:02:54,421 - lightrag - INFO - Storage Initialization complet

CancelledError: 

INFO:openai._base_client:Retrying request to /chat/completions in 0.447377 seconds
INFO:openai._base_client:Retrying request to /chat/completions in 0.451209 seconds
INFO:openai._base_client:Retrying request to /chat/completions in 0.469781 seconds
INFO:openai._base_client:Retrying request to /chat/completions in 0.448269 seconds


Error during LM Studio generation: Error code: 400 - {'error': 'Model unloaded.'}
Error during LM Studio generation: Error code: 400 - {'error': 'Model unloaded.'}
Error during LM Studio generation: Error code: 400 - {'error': 'Model unloaded.'}
Error during LM Studio generation: Error code: 400 - {'error': 'Model unloaded.'}
Error during LM Studio generation: Error code: 404 - {'error': {'message': 'Failed to load model "google/gemma-3-1b". Error: Model does not exist.', 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_found'}}
Error during LM Studio generation: Error code: 404 - {'error': {'message': 'Failed to load model "google/gemma-3-1b". Error: Model does not exist.', 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_found'}}
Error during LM Studio generation: AsyncCompletions.create() got an unexpected keyword argument 'history_messages'
Error during LM Studio generation: Error code: 404 - {'error': {'message': 'Failed to load model "goo

...

INFO:openai._base_client:Retrying request to /chat/completions in 0.447489 seconds
INFO:openai._base_client:Retrying request to /chat/completions in 0.381349 seconds


Error during LM Studio generation: Connection error.
Error during LM Studio generation: Connection error.
Error during LM Studio generation: AsyncCompletions.create() got an unexpected keyword argument 'history_messages'
2025-07-07 16:39:40,320 - lightrag - INFO - Chunk 2287 of 3990 extracted 0 Ent + 0 Rel


INFO:openai._base_client:Retrying request to /chat/completions in 0.490692 seconds
INFO:openai._base_client:Retrying request to /chat/completions in 0.926267 seconds
INFO:openai._base_client:Retrying request to /chat/completions in 0.764571 seconds
INFO:openai._base_client:Retrying request to /chat/completions in 0.447424 seconds


Error during LM Studio generation: Connection error.


INFO:openai._base_client:Retrying request to /chat/completions in 0.962528 seconds
INFO:openai._base_client:Retrying request to /chat/completions in 0.877867 seconds
